2025-11-19

find all references (högerklicka) i VS-code - hitta alla referenser


- Load data - Första versionen - hämtar ut ny information från nätet (stocks-API, från url) Kan bli problem, kan få "rate-limiting" om man gör det för mycket. 

- Andra - lägger till en lokal cache, sparar lokalt på disk ("Stocksdatalocal")

- tredje . spara lokalt hos klienten(användaren) istället (dcc.store - dynamisk komponent)
- lagra i webbläsare (cookie). Hur komma åt sparad information i webbläsaren - lägger till som input, filter DF, följer med in när vi anropar highest lowes value - "ohlc-radio". 
- Ska vara json-objekt som sparas i webbläsare (cookies) - 

- Sliders - 

- main - dashapp

- layout - finns att hitta på nätet. 

Load data - class eller metoder? 
@callback 





In [ ]:
import dash
print(dash.__version__)


https://dashboard.render.com/

Onrender.com deploya på  
deploy som cloudtjänst   
   
Ta bort server i run_server (main)  
   
Olika sätt att deploya.... L5.1-L5.4  
   
dash-api.py - direkt  Dashboard attempt 1, callback using GET request to API  
   
Lagra data i klientens browser - dcc.Store (json)  
  
Koppla till github (onrender) 

Date-time  , tidsangivelser, komplicerade och jätteviktiga.  komplicerat med tidszoner och sommar/vintertid. Hela internet ett tidsdelande system, tidsstämplarna. 

ChatGPT sammanställning: 

Absolutely — här är en tydlig “från 0 till deploy”-checklista som passar din kodbas och Render (onrender) + VS Code. Jag har även lagt med färdiga filer du kan ladda ned och lägga i repot.

# 1) Projektstruktur (minimum)

```
your-repo/
├─ main.py
├─ layout.py
├─ load_data.py
├─ time_filtering.py
├─ stocksdata/
│  ├─ AAPL_TIME_SERIES_DAILY_ADJUSTED.csv
│  ├─ AAPL_TIME_SERIES_INTRADAY_EXTENDED.csv
│  ├─ NVDA_TIME_SERIES_DAILY_ADJUSTED.csv
│  └─ ... (motsv. för NVDA/TSLA/IBM)
├─ requirements.txt
├─ Procfile
└─ render.yaml   (valfri men smidig på Render)
```

Obs: `main.py` väntar att data ligger i mappen `stocksdata/` (se `path = os.path.join(directory_path, "stocksdata")` i koden). Lägg in dina CSV:er där.

# 2) Snabb fix i koden (2 små buggar)

1. I `main.py`
   Byt importen till den korrekta:

```python
# FEL: import plotly_express as px
import plotly.express as px
```

2. I `time_filtering.py`
   Funktionen tar första datumet (äldsta) istället för senaste. Ändra sista-dag-rad:

```python
last_day = df.index.max().date()
```

# 3) Lokalt i VS Code

1. Skapa och aktivera virtuell miljö

```bash
python -m venv .venv
# Windows: .venv\Scripts\activate
# macOS/Linux:
source .venv/bin/activate
```

2. Installera beroenden

```bash
pip install -r requirements.txt
```

3. Kör appen

```bash
python main.py
```

Appen startar på [http://127.0.0.1:8050](http://127.0.0.1:8050) (debug-läge är på i `if __name__ == "__main__":`).

Tips i VS Code:

* Jag har lagt en färdig startkonfiguration (se nedan); öppna kommandopaletten → “Debug: Start Debugging” → “Run Dash (main.py)”.

# 4) Git & GitHub

1. Initiera repo (om du inte redan har)

```bash
git init
git add .
git commit -m "Initial Dash app"
```

2. Skapa ett nytt GitHub-repo och pusha:

```bash
git remote add origin <din-github-url>
git branch -M main
git push -u origin main
```

# 5) Render (onrender) — deploy

Variant A (GUI):

1. Logga in på Render → “New +” → “Web Service”.
2. Välj ditt GitHub-repo.
3. Environment: **Python**
   Build command: `pip install -r requirements.txt`
   Start command: `gunicorn main:server`
4. Plan: Free (eller som du vill) → “Create Web Service”.

Variant B (render.yaml – One-click):

* Om du committar filen `render.yaml` i repo-roten kommer Render läsa in rätt bygg- och startkommandon automatiskt när du skapar tjänsten.

# 6) Filer jag genererade åt dig (klara att använda)

Ladda ner och lägg i din projektrot (de är generiska och funkar direkt med din kod):

* [requirements.txt](sandbox:/mnt/data/requirements.txt)
  Innehåll: `dash`, `dash-bootstrap-components`, `plotly`, `pandas`, `python-dateutil`, `requests`, `gunicorn`.

* [Procfile](sandbox:/mnt/data/Procfile)
  Innehåll: `web: gunicorn main:server`

* [render.yaml](sandbox:/mnt/data/render.yaml) *(valfri men bekväm)*

* [.gitignore](sandbox:/mnt/data/.gitignore)

* [.vscode/launch.json](sandbox:/mnt/data/.vscode/launch.json) *(starta appen direkt från VS Code Debug)*

# 7) Vanliga fallgropar (snabbcheck)

* 404 eller vit sida på Render? Kontrollera att startkommandot är exakt `gunicorn main:server` och att `server = app.server` finns i `main.py` (det gör det).
* Saknade CSV-filer? Säkerställ att `stocksdata/` finns med i repot (inte bara lokalt) och att filnamnen matchar exakt det som `load_data.py` förväntar sig.
* `plotly_express`-ImportError → se fix i punkt 2.
* Timeout vid datahämtning från API: Du använder lokala CSV:er via `StockDataLocal`, så API-nyckel krävs inte. (Om du byter till `StockDataAPI`, lägg till env-variabel med din Alpha Vantage-nyckel och anropa den klassen i stället.)

# 8) Extra polish (frivilligt)

* Lägg till enklare titelformat och axel-etiketter i `update_graph` för en snyggare graf.
* Sätt `debug=False` när du kör i produktion (på Render körs `gunicorn`, så debug är redan av, men nämnvärt om du nånsin kör Flask servern direkt i produktion).

Vill du att jag patchar filerna åt dig (t.ex. uppdaterar importen och tidfiltret) och ger dig en färdig zip? Säg till så fixar jag det direkt.


**time_filtering.py**  
  
AI25-Databehandling/Lectures/L5-stock_dashboard/L5.2/time_filtering.py

In [ ]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

def filter_time(df, days = 0):
    last_day = df.index[0].date()
    start_day = last_day-relativedelta(days = days)
    df = df.sort_index().loc[start_day:last_day]
    return df 

**style.css**  
  
AI25-Databehandling/Lectures/L5-stock_dashboard/L5.2/assets/style.css

In [ ]:
span{
    font-size: 1rem;
}

h1{
    color: blueviolet;
}

**data_gathering.ipynb**  
    
AI25-Databehandling/Lectures/L5-stock_dashboard/data_gathering.ipynb

In [ ]:
import time
import pandas as pd 


symbols = ["AAPL", "NVDA", "TSLA", "IBM"]
stock_list = []

for symbol in symbols:
    for function, time_column in zip(["TIME_SERIES_DAILY_ADJUSTED", "TIME_SERIES_INTRADAY_EXTENDED"], ["timestamp", "time"]):
        url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}&datatype=csv&outputsize=full&interval=60min"

        df = pd.read_csv(url)
        df.set_index(df[time_column], inplace=True)
        df.index = pd.to_datetime(df.index)
        df.drop(time_column, axis=1, inplace=True)
        df.to_csv(f"../data/Stocksdata/{symbol}_{function}.csv")

        # to not overload API
        time.sleep(15)

**load_data.py**  
    
AI25-Databehandling/Code-alongs/l5.dash/load_data.py  
Hämtar informationen från Alpha Vantage  
Skaffa en egen API-nyckel, denna kan sluta fungera närsomhelst 

In [ ]:
import requests
import pandas as pd
import os

class StockDataAPI:
    """Class with methods to get and process data from Alpha Vantage.
    """

    def __init__(self, api_key, data_function = "TIME_SERIES_DAILY"):
        """
        Args:
            api_key: API key from alphavantage
        """
        self._data_function = data_function
        self._api_key = api_key
    
    def get_stock(self, symbol):
        url = f"https://www.alphavantage.co/query?function={self._data_function}&symbol={symbol}&apikey={self._api_key}&outputsize=full"
        try:
            data = requests.get(url).json()
            df = pd.DataFrame(data["Time Series (Daily)"]).transpose().astype(float)
            df.index = pd.to_datetime(df.index)
            df.columns = ["Open", "High", "Low", "Close", "Volume"]
            return df
        except KeyError as err:
            import pickle
            with open("Data/stocks.pckl", "rb") as file:
                disk_data = pickle.load(file)
            disk_data = pd.concat(disk_data, axis=1)
            data = disk_data[symbol]
            df = pd.DataFrame(data)
            df.index = pd.to_datetime(df.index)
            df.columns = ["Close"]
            return df

class StockDataLocal:
    def __init__(self, data_folder = "Data"):
        self._data_folder = data_folder

    def get_dataframe(self, stockname):
        stock_df = []
        for path_ending in ["_TIME_SERIES_DAILY_ADJUSTED.csv",
                     "_TIME_SERIES_INTRADAY_EXTENDED.csv"]:
            path = os.path.join(self._data_folder, stockname+path_ending)
            stock = pd.read_csv(path, index_col=0, parse_dates=True)
            stock.index.rename("Date", inplace=True)
            stock_df.append(stock)
        return stock_df

def run_test():
    api_key = os.getenv("ALPHA_API_KEY")
    test = StockDataAPI(api_key)
    print(f"Result: {test.get_stock("AAPL")}")

if __name__ == '__main__':
    run_test()

**main.py**  
  
AI25-Databehandling/Lectures/L5-stock_dashboard/L5.2/main.py

In [ ]:
from dash import dcc, html
from dash.dependencies import Output, Input
import dash
from load_data import StockDataLocal
import plotly_express as px
from time_filtering import filter_time
import os

# module variables

path = os.path.join(os.path.dirname(__file__), "stocksdata")

# create an object of type StockDataLocal so that we can load data
stock_data_object = StockDataLocal(path)


symbol_dict = {"AAPL": "Apple", "NVDA": "Nvidia",
               "TSLA": "Tesla", "IBM": "IBM"}

# stock_symbol: [daily_df, intradaily_df]
df_dict = {symbol: stock_data_object.stock_dataframe(
    symbol) for symbol in symbol_dict}


stock_options_dropdown = [{"label": name, "value": symbol}
                          for symbol, name in symbol_dict.items()]

# ohlc - Open, High, Low, Close
ohlc_options = [{"label": option.capitalize(), "value": option}
                for option in ["open", "high", "low", "close"]]

slider_marks = {i: mark for i, mark in enumerate(
    ["1 day", "1 week", "1 month", "3 months", "1 year", "5 year", "Max"])}

# creates a Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Stocks viewer"),
    html.P("Choose a stock"),
    dcc.Dropdown(id='stock-picker-dropdown', className='',
                 options=stock_options_dropdown,
                 value='AAPL',
                 placeholder='Apple'
                 ),
    dcc.RadioItems(id='ohlc-radio', className='',
        options=ohlc_options,
        value='close'
    ),
    dcc.Graph(id="stock-graph"),
    dcc.Slider(id='time-slider',
               min=0,
               max=6,
               step=None,
               value=2,
               marks=slider_marks
               )
])

# when something changes in the input component, the code in function below will run and update the output component
# the components are connected through their id


@app.callback(
    Output("stock-graph", "figure"),
    Input("stock-picker-dropdown", "value"),
    Input("ohlc-radio", "value"),
    Input("time-slider", "value")
)
def update_graph(stock, ohlc, time_index):
    """Updates graph based on different unputs"""
    # tuple unpack the list in the dict
    dff_daily, dff_intraday = df_dict[stock]

    dff = dff_intraday if time_index <= 2 else dff_daily

    days = {i: day for i, day in enumerate([1, 7, 30, 90, 365, 365*5])}

    dff = dff if time_index == 6 else filter_time(dff, days=days[time_index])

    fig = px.line(dff, x=dff.index,
                  y=ohlc, title=symbol_dict[stock])

    # TODO: Add figure settings to make graph look nice

    return fig


if __name__ == "__main__":
    app.run_server(debug=True)